# Supplementary Results 8
## Sample-by-Sample Detection Evaluation 

In [2]:
import copy

import pandas as pd
from plotly.subplots import make_subplots
import plotly.io as pio

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.sample_metrics as sm

pio.renderers.default = "browser"

### Load Data

In [3]:
global_measures = sm.load_global_metrics(DATASET_NAME, PROCESSED_DATA_DIR, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
global_measures.drop(index=[peyes.constants.ACCURACY_STR, peyes.constants.BALANCED_ACCURACY_STR], inplace=True)    # Drop Acc+Balanced-Acc metrics

fixation_sdt = sm.load_sdt(DATASET_NAME, PROCESSED_DATA_DIR, label=1, stimulus_type=STIMULUS_TYPE, metric=None, iteration=1)
fixation_sdt = fixation_sdt.loc[[peyes.constants.D_PRIME_STR, peyes.constants.F1_STR]]      # Keep only d' and f1 metrics
fixation_sdt = fixation_sdt.rename(index=lambda idx: f"fixation_{idx}")     # Rename index

saccade_sdt = sm.load_sdt(DATASET_NAME, PROCESSED_DATA_DIR, label=2, stimulus_type=STIMULUS_TYPE, metric=None)
saccade_sdt = saccade_sdt.loc[[peyes.constants.D_PRIME_STR, peyes.constants.F1_STR]]        # Keep only d' and f1 metrics
saccade_sdt = saccade_sdt.rename(index=lambda idx: f"saccade_{idx}")        # Rename index

fixation_saccade_sdt = pd.concat([fixation_sdt, saccade_sdt], axis=0)

### Fixation & Saccade Detection
Evaluate performance by measuring how well the detector detects _fixation-samples_ or _saccade-samples_ out of all samples.  
Evaluation is based on _Discriminability Index_ ($d'$) and _f1-score_

In [4]:
sdt_statistics, sdt_pvalues, sdt_nemenyi, sdt_Ns = sm.friedman_nemenyi(fixation_saccade_sdt, [GT1, GT2])

sdt_pvalues <= ALPHA

gt,MN,RA
metric,,
fixation_d_prime,True,True
fixation_f1,True,True
saccade_d_prime,True,True
saccade_f1,True,True


In [5]:
pd.concat([sdt_statistics, sdt_pvalues], axis=1, keys=['Q', 'p']).stack(1, future_stack=True)

Q             p
metric           gt                         
fixation_d_prime MN  55.517419  3.644007e-10
                 RA  79.855335  3.827345e-15
fixation_f1      MN  67.749677  1.182213e-12
                 RA  98.343580  5.559357e-19
saccade_d_prime  MN  55.403599  3.842169e-10
                 RA  71.509434  2.003739e-13
saccade_f1       MN  51.455013  2.399829e-09
                 RA  87.121294  1.199342e-16

#### Post Hoc Analysis

In [6]:
post_hoc_fix_dprime = sm.post_hoc_table(sdt_nemenyi, f"fixation_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_fix_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *    n.s.    n.s.    n.s.       **
         RA      --    n.s.       *    n.s.    n.s.    n.s.      ***
ivvt     MN  0.9974      --    n.s.    n.s.    n.s.    n.s.        *
         RA  0.9747      --    n.s.    n.s.    n.s.    n.s.       **
idt      MN  0.0184  0.1216      --    n.s.    n.s.    n.s.     n.s.
         RA  0.0439  0.4048      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.0786  0.3260  0.9995      --    n.s.    n.s.     n.s.
         RA  0.1540  0.7058  0.9995      --    n.s.    n.s.     n.s.
engbert  MN  0.9996  1.0000  0.0774  0.2355      --    n.s.        *
         RA  0.9968  1.0000  0.2325  0.5049      --    n.s.       **
nh       MN  0.2508  0.6398  0.9756  0.9994  0.5264      --     n.s.
         RA  0.0768  0.5295  1.0000  1.0000  0.3337      --     n.s.
remodnav MN  0.0026  0.0285  0.9993  0.9741  0.0158  0.8274       --
         RA  0.0001  0.0060  0.7920  0.5027  0.0016  0.6809       --

In [7]:
post_hoc_fix_f1 = sm.post_hoc_table(sdt_nemenyi, f"fixation_{peyes.constants.F1_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_fix_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.      ***
         RA      --    n.s.    n.s.    n.s.    n.s.    n.s.      ***
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.      ***
         RA  0.9999      --    n.s.    n.s.    n.s.    n.s.      ***
idt      MN  0.4603  0.3699      --    n.s.    n.s.    n.s.     n.s.
         RA  0.3985  0.2200      --    n.s.    n.s.    n.s.        *
idvt     MN  0.5512  0.4565  1.0000      --    n.s.    n.s.     n.s.
         RA  0.4673  0.2726  1.0000      --    n.s.    n.s.        *
engbert  MN  0.9882  0.9958  0.0824  0.1182      --    n.s.      ***
         RA  0.9951  0.9999  0.0890  0.1177      --    n.s.      ***
nh       MN  0.9649  0.9339  0.9644  0.9829  0.5911      --       **
         RA  0.9808  0.9143  0.9132  0.9431  0.7418      --      ***
remodnav MN  0.0001  0.0000  0.1894  0.1380  0.0000  0.0086       --
         RA  0.0000  0.0000  0.0289  0.0201  0.0000  0.0001       --

In [8]:
post_hoc_sac_dprime = sm.post_hoc_table(sdt_nemenyi, f"saccade_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_sac_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.    n.s.    n.s.    n.s.    n.s.        †
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  1.0000      --    n.s.    n.s.    n.s.    n.s.        †
idt      MN  0.4143  0.3963      --    n.s.      **    n.s.     n.s.
         RA  0.1776  0.1803      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.6619  0.6435  0.9999      --      **    n.s.     n.s.
         RA  0.5328  0.5373  0.9974      --     ***    n.s.     n.s.
engbert  MN  0.5949  0.6137  0.0016  0.0079      --    n.s.       **
         RA  0.3655  0.3614  0.0000  0.0007      --    n.s.      ***
nh       MN  1.0000  1.0000  0.3927  0.6398  0.6175      --     n.s.
         RA  1.0000  1.0000  0.1383  0.4618  0.4325      --        *
remodnav MN  0.3856  0.3681  1.0000  0.9997  0.0013  0.3647       --
         RA  0.0623  0.0635  0.9998  0.9624  0.0000  0.0453       --

In [9]:
post_hoc_sac_f1 = sm.post_hoc_table(sdt_nemenyi, f"saccade_{peyes.constants.F1_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_sac_f1

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.      **       *    n.s.    n.s.     n.s.
         RA      --    n.s.     ***     ***    n.s.    n.s.     n.s.
ivvt     MN  1.0000      --       *       *    n.s.    n.s.     n.s.
         RA  1.0000      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.0080  0.0113      --    n.s.      **    n.s.     n.s.
         RA  0.0000  0.0001      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.0146  0.0201  1.0000      --       *    n.s.     n.s.
         RA  0.0001  0.0001  1.0000      --     ***    n.s.     n.s.
engbert  MN  1.0000  1.0000  0.0063  0.0116      --    n.s.     n.s.
         RA  1.0000  1.0000  0.0000  0.0000      --    n.s.     n.s.
nh       MN  0.8752  0.9085  0.3409  0.4399  0.8486      --     n.s.
         RA  0.5831  0.6163  0.0952  0.1440  0.4498      --     n.s.
remodnav MN  0.7356  0.7869  0.5188  0.6250  0.6977  1.0000       --
         RA  0.1977  0.2200  0.3849  0.4905  0.1238  0.9967       --

## Original Figure

In [10]:
sdt_metrics_fig = sm.sdt_distributions_figure(
    fixation_saccade_sdt,
    GT1, GT2,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    only_box=False,
    show_other_gt=True,
    share_x=True,
)

sdt_metrics_fig.show()

## Final Fig:
#### (1) Only Show d'
#### (2) Show RA and MN Separately

In [11]:
NAME = "supp_fig_E1"
WIDTH, HEIGHT = 1200, 800

ROW_TITLES, COL_TITLES = ["GT: <i>RA</i>", "GT: <i>MN</i>"], ["Fixations", "Saccades"]

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=25, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=22, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)

In [12]:
fig = make_subplots(
    rows=len(ROW_TITLES), cols=len(COL_TITLES),
    row_titles=ROW_TITLES, column_titles=COL_TITLES,
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.05, horizontal_spacing=0.05,
)

for trace in sdt_metrics_fig.data:
    if trace["scalegroup"].endswith('f1'):
        continue
    new_tr = copy.deepcopy(trace)
    gt1 = new_tr["name"].split(",")[0]
    gt2 = "MN" if gt1=="RA" else "RA"

    # convert from violin plot to ridge plot by assigning the `y0` prop and removing `x0` prop
    if new_tr["x0"] == "Other GT":
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = f"Ann. {gt2}"
    elif new_tr["x0"].startswith("i"):
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
    elif new_tr["x0"] == "remodnav":
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
    else:
        new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
    new_tr['x'] = new_tr['y']

    # reset irrelevant props
    new_tr['y'] = new_tr['x0'] = None
    new_tr["showlegend"] = new_tr['points'] = False
    new_tr["side"] = "positive"
    new_tr["box"] = new_tr["meanline"] = None
    new_tr["width"] = 2
    new_tr["opacity"] = 1
    new_tr["visible"] = True
    new_tr["scalegroup"] = new_tr["scalegroup"].split("_")[0]

    # add the trace to the correct subplot
    r = 1 if gt1=="RA" else 2
    c = 1 if new_tr["scalegroup"].startswith("fixation") else 2
    fig.add_trace(new_tr, row=r, col=c)

# update x axes
fig.for_each_xaxis(lambda xax: xax.update(
    showline=False, zeroline=False, showgrid=False, gridcolor='lightgray', gridwidth=1, tickfont=AXIS_TICK_FONT,
))
for c in range(len(COL_TITLES)):
    fig.update_xaxes(row=2, col=c+1, title=dict(text=r"$d'$", font=AXIS_LABEL_FONT, standoff=2),)

# update y axes
fig.for_each_yaxis(lambda yax: yax.update(
    showline=False, zeroline=False, showgrid=True, gridcolor='lightgray', gridwidth=1, tickfont=AXIS_TICK_FONT,
))
for r in range(len(ROW_TITLES)):
    fig.update_yaxes(row=r+1, col=1, title=dict(text="Detector", font=AXIS_LABEL_FONT, standoff=4),)

# update subtitles
fig.for_each_annotation(lambda ann: ann.update(
    font=TITLE_FONT, textangle=0,
    xref='paper', xanchor='center', x=0.49 if ann.text in ROW_TITLES else ann.x,
    yref='paper', yanchor='top', y=1.0 if ann.text==ROW_TITLES[0] else 0.475 if ann.text==ROW_TITLES[1] else 1.05,
))

fig.update_layout(
    font_family=FONT_FAMILY,
    width=WIDTH, height=HEIGHT,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=27.5, b=45, pad=0),
    showlegend=False,
)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"), indent=4)
fig.show()

## Repeat Analysis
### Repeating the analysis with subset of image-stimulus trials that were recorded @ 500Hz

In [13]:
subset = fixation_saccade_sdt.drop(columns=NON_500HZ_TRIALS, level=peyes.constants.TRIAL_ID_STR)

subset_statistics, subset_pvalues, subset_nemenyi, subset_Ns = sm.friedman_nemenyi(fixation_saccade_sdt, [GT1, GT2])
pd.concat([sdt_statistics, sdt_pvalues, subset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)

Q             p  is_sig
metric           gt                                 
fixation_d_prime MN  55.517419  3.644007e-10    True
                 RA  79.855335  3.827345e-15    True
fixation_f1      MN  67.749677  1.182213e-12    True
                 RA  98.343580  5.559357e-19    True
saccade_d_prime  MN  55.403599  3.842169e-10    True
                 RA  71.509434  2.003739e-13    True
saccade_f1       MN  51.455013  2.399829e-09    True
                 RA  87.121294  1.199342e-16    True

In [14]:
subset_post_hoc_fix_dprime = sm.post_hoc_table(subset_nemenyi, f"fixation_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
subset_post_hoc_fix_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *    n.s.    n.s.    n.s.       **
         RA      --    n.s.       *    n.s.    n.s.    n.s.      ***
ivvt     MN  0.9974      --    n.s.    n.s.    n.s.    n.s.        *
         RA  0.9747      --    n.s.    n.s.    n.s.    n.s.       **
idt      MN  0.0184  0.1216      --    n.s.    n.s.    n.s.     n.s.
         RA  0.0439  0.4048      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.0786  0.3260  0.9995      --    n.s.    n.s.     n.s.
         RA  0.1540  0.7058  0.9995      --    n.s.    n.s.     n.s.
engbert  MN  0.9996  1.0000  0.0774  0.2355      --    n.s.        *
         RA  0.9968  1.0000  0.2325  0.5049      --    n.s.       **
nh       MN  0.2508  0.6398  0.9756  0.9994  0.5264      --     n.s.
         RA  0.0768  0.5295  1.0000  1.0000  0.3337      --     n.s.
remodnav MN  0.0026  0.0285  0.9993  0.9741  0.0158  0.8274       --
         RA  0.0001  0.0060  0.7920  0.5027  0.0016  0.6809       --

In [15]:
subset_post_hoc_sac_dprime = sm.post_hoc_table(subset_nemenyi, f"saccade_{peyes.constants.D_PRIME_STR}", [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
subset_post_hoc_sac_dprime

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.    n.s.    n.s.    n.s.    n.s.        †
ivvt     MN  1.0000      --    n.s.    n.s.    n.s.    n.s.     n.s.
         RA  1.0000      --    n.s.    n.s.    n.s.    n.s.        †
idt      MN  0.4143  0.3963      --    n.s.      **    n.s.     n.s.
         RA  0.1776  0.1803      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.6619  0.6435  0.9999      --      **    n.s.     n.s.
         RA  0.5328  0.5373  0.9974      --     ***    n.s.     n.s.
engbert  MN  0.5949  0.6137  0.0016  0.0079      --    n.s.       **
         RA  0.3655  0.3614  0.0000  0.0007      --    n.s.      ***
nh       MN  1.0000  1.0000  0.3927  0.6398  0.6175      --     n.s.
         RA  1.0000  1.0000  0.1383  0.4618  0.4325      --        *
remodnav MN  0.3856  0.3681  1.0000  0.9997  0.0013  0.3647       --
         RA  0.0623  0.0635  0.9998  0.9624  0.0000  0.0453       --